# BQ Compression – Load Data and compress vectors

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY[:20]}")
print(f"OpenAI URL: {OPENAI_URL}")

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

## Create Collection with BQ configuration

[Docs: Binary Quantization (BQ)](https://weaviate.io/developers/weaviate/configuration/compression/bq-compression)

Note #1: Binary Quantization works from the first object added to the collection. No training required.

Note #2: Binary Quantization works both with HNSW and Flat index.

In [ ]:
from weaviate.classes.config import Configure, VectorDistances

client.collections.delete("WikiQ")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="WikiQ",

    vector_config=[
        Configure.Vectors.text2vec_openai(
            name="main_vector",

            model="text-embedding-3-small",
            base_url=OPENAI_URL,
            source_properties=['title', 'text'],

            # Configure BQ with flat vector index
            vector_index_config=Configure.VectorIndex.flat(
                distance_metric=VectorDistances.COSINE,
                vector_cache_max_objects=100_000,
                quantizer=Configure.VectorIndex.Quantizer.bq(
                    rescore_limit=200,
                    cache=True
                )
            ),

            # HSNW example
            # vector_index_config=Configure.VectorIndex.hsnw(
            #     quantizer=Configure.VectorIndex.Quantizer.bq(
            #         rescore_limit=200,
            #         cache=True
            #     )
            # ),
        )
    ],
)

## The rest is the same

In [ ]:
from data_loader import import_wiki_data
import_wiki_data(client, "WikiQ", 25000)

In [ ]:
wikiQ = client.collections.get("WikiQ")
wikiQ.aggregate.over_all()

## Clean up

In [ ]:
# client.collections.delete("WikiQ")

## Close the client

In [ ]:
client.close()